In [19]:
# 1. 문제 정의
# 평가: roc-auc
# target: Outcome
# 최종 파일: result.csv(컬럼 1개 pred, 1 확률값)
# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("diabetes_train.csv")
test = pd.read_csv("diabetes_test.csv")

# 3. 탐색적 데이터분석(EDA)
print("===== 데이터 크기 =====")
print("Train Shape", train.shape)
print("Test Shape", test.shape)
print("\n")  # 줄 바꿈

print("===== train 데이터 샘플 =====")
print(train.head(1))
print("\n")

print("===== test 데이터 샘플 =====")
print(test.head(1))
print("\n")

print("===== 데이터 정보(자료형) =====")
print(train.info())
print("\n")

print("===== train 결측치 수 =====")
print(train.isnull().sum().sum())
print("\n")

print("===== test 결측치 수 =====")
print(test.isnull().sum().sum())
print("\n")

print("===== target 빈도 =====")
print(train['Outcome'].value_counts())

===== 데이터 크기 =====
Train Shape (614, 9)
Test Shape (154, 8)


===== train 데이터 샘플 =====
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            1      118             58             36       94  33.3   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.261   23        0  


===== test 데이터 샘플 =====
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            3      102             74              0        0  29.5   

   DiabetesPedigreeFunction  Age  
0                     0.121   32  


===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               614 non-null    int64  
 1   Glucose                   614 non-null    int64  
 2   BloodPressure             614 non-null    int64  
 3   SkinThickness             614

In [21]:
# 4. 데이터 전처리
target = train.pop('Outcome')

# 5. 검증 데이터 나누기
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

print("\n ===== 분할된 데이터 크기 =====")
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)

print("\n ===== 예측 결과 확인 (샘플 5개) =====")
print(pred[:5])

from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print("\n roc_auc:", roc_auc)

# 7. 예측 및 결과 파일 생성
pred = rf.predict_proba(test)
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv", index=False)

# 제출 파일 확인
print("\n ===== 제출파일 (샘플 5개) =====")
print(pd.read_csv("result.csv").head())


 ===== 분할된 데이터 크기 =====
(491, 8) (123, 8) (491,) (123,)

 ===== 예측 결과 확인 (샘플 5개) =====
[[0.88 0.12]
 [0.29 0.71]
 [0.41 0.59]
 [0.75 0.25]
 [0.89 0.11]]

 roc_auc: 0.8002739726027398

 ===== 제출파일 (샘플 5개) =====
   pred
0  0.17
1  0.33
2  0.11
3  0.04
4  0.09


### 성능 개선

In [32]:
# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("diabetes_train.csv")
test = pd.read_csv("diabetes_test.csv")

# 4. 데이터 전처리
# 스케일링
target = train.pop('Outcome')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

# 5. 검증 데이터 나누기
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=5, n_estimators=500, random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)

from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print("\n roc_auc:", roc_auc)

# 7. 예측 및 결과 파일 생성
pred = rf.predict_proba(test)
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv", index=False)


 roc_auc: 0.8246575342465753
